In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

from utils import mnist

In [4]:
train_loader, test_loader = mnist()

Extracting ./MNIST_data\MNIST\raw\train-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw


Extracting ./MNIST_data\MNIST\raw\train-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw



Extracting ./MNIST_data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./MNIST_data\MNIST\raw


Extracting ./MNIST_data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./MNIST_data\MNIST\raw
Processing...
Done!

..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [11]:
class Net(nn.Module):
    def __init__(self, log_softmax=False):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)
        self.log_softmax = log_softmax
        self.optim = optim.SGD(self.parameters(), lr=1.0)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = torch.sigmoid(self.fc1(x))
        x = self.fc2(x)
        if self.log_softmax:
            x = F.log_softmax(x, dim=1)
        else:
            x = torch.log(F.softmax(x, dim=1))
        return x
    
    def loss(self, output, target, **kwargs):
        self._loss = F.nll_loss(output, target, **kwargs)
        return self._loss

In [12]:
def train(epoch, models):
    for batch_idx, (data, target) in enumerate(train_loader):
        for model in models:
            model.optim.zero_grad()
            output = model(data)
            loss = model.loss(output, target)
            loss.backward()
            model.optim.step()
            
        if batch_idx % 200 == 0:
            line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader))
            losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
            print(line + losses)
            
    else:
        batch_idx += 1
        line = 'Train Epoch: {} [{}/{} ({:.0f}%)]\tLosses '.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader))
        losses = ' '.join(['{}: {:.6f}'.format(i, m._loss.item()) for i, m in enumerate(models)])
        print(line + losses)

In [13]:
models = [Net(), Net(True)]

In [14]:
avg_lambda = lambda l: 'Loss: {:.4f}'.format(l)
acc_lambda = lambda c, p: 'Accuracy: {}/{} ({:.0f}%)'.format(c, len(test_loader.dataset), p)
line = lambda i, l, c, p: '{}: '.format(i) + avg_lambda(l) + '\t' + acc_lambda(c, p)

def test(models):
    test_loss = [0]*len(models)
    correct = [0]*len(models)
    with torch.no_grad():
        for data, target in test_loader:
            output = [m(data) for m in models]
            for i, m in enumerate(models):
                test_loss[i] += m.loss(output[i], target, reduction='sum').item() # sum up batch loss
                pred = output[i].data.max(1, keepdim=True)[1] # get the index of the max log-probability
                correct[i] += pred.eq(target.data.view_as(pred)).cpu().sum()
    
    for i in range(len(models)):
        test_loss[i] /= len(test_loader.dataset)
    correct_pct = [100. * c / len(test_loader.dataset) for c in correct]
    lines = '\n'.join([line(i, test_loss[i], correct[i], correct_pct[i]) for i in range(len(models))]) + '\n'
    report = 'Test set:\n' + lines
    
    print(report)

In [15]:
for epoch in range(2):
    train(epoch, models)
    test(models)

Train Epoch: 0 [0/60000 (0%)]	Losses 0: 2.359224 1: 2.307634
Train Epoch: 0 [10000/60000 (17%)]	Losses 0: 0.122188 1: 0.097579
Train Epoch: 0 [20000/60000 (33%)]	Losses 0: 0.125888 1: 0.136355
Train Epoch: 0 [30000/60000 (50%)]	Losses 0: 0.100993 1: 0.095209
Train Epoch: 0 [40000/60000 (67%)]	Losses 0: 0.037869 1: 0.050014
Train Epoch: 0 [50000/60000 (83%)]	Losses 0: 0.175265 1: 0.187708
Train Epoch: 0 [60000/60000 (100%)]	Losses 0: 0.123824 1: 0.078190
Test set:
0: Loss: 0.1306	Accuracy: 9602/10000 (96%)
1: Loss: 0.1325	Accuracy: 9587/10000 (96%)

Train Epoch: 1 [0/60000 (0%)]	Losses 0: 0.028824 1: 0.039987
Train Epoch: 1 [10000/60000 (17%)]	Losses 0: 0.165533 1: 0.161263
Train Epoch: 1 [20000/60000 (33%)]	Losses 0: 0.114608 1: 0.073211
Train Epoch: 1 [30000/60000 (50%)]	Losses 0: 0.042947 1: 0.041924
Train Epoch: 1 [40000/60000 (67%)]	Losses 0: 0.074710 1: 0.060629
Train Epoch: 1 [50000/60000 (83%)]	Losses 0: 0.082692 1: 0.043531
Train Epoch: 1 [60000/60000 (100%)]	Losses 0: 0.114803